In [2]:
import pandas as pd
import numpy as np
france = pd.read_excel('France.xlsx') 
#importing the raw French data from IMF

FileNotFoundError: [Errno 2] No such file or directory: 'France.xlsx'

In [ ]:
US = pd.read_excel('USCPI.xlsx')
#importing the US CPI data, ready to merge the 2 dataframes

In [ ]:
merged_df = pd.concat([france, US], axis=1)
#merged the 2 dataframes such that the US CPI column is added
pd.options.display.max_rows = None
#changing the display so all rows are displayed

In [ ]:
import numpy as np
#this is to take the natural log of the exchange rate column.
merged_df['(i) France: Natural log of the Exchange Rate per U.S. Dollar'] = np.log(merged_df['France: Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate'])    
merged_df

In [ ]:
#to find the difference between a month and the following month, 
#We took the column and duplicated it - with a small difference - I shifted every row down by one position. 
#Let us call the original column 'A' and this duplicated column A but with every row down one position 'B'
#Therefore, if the first data entry from A is in row 0, it is now in row 1 of B
merged_df["Shifted Natural log of the Exchange Rate per U.S. Dollar"] = merged_df["(i) France: Natural log of the Exchange Rate per U.S. Dollar"].shift(1)
#We then created a new column, one for the difference between the consecutive months. 
#We subtracted the original column values from the shifted ones. this gives the change from one month to the next
#For example: if row 1 from A contains the data for month 2, row 1 from B contains the data for month 1. 
#Therefore, subtracting them gives the monthly change, the difference.
merged_df["(ii) Monthly difference, Natural log of the Exchange Rate per U.S. Dollar"] = merged_df["(i) France: Natural log of the Exchange Rate per U.S. Dollar"] - merged_df["Shifted Natural log of the Exchange Rate per U.S. Dollar"]
merged_df
#row 0 gives a NaN value since the data for the month 0 (ie December 1959), the one before Jan 1960 where our data starts, does not exist/is not included in the dataset.

In [ ]:
#Following a similar process to finding the monthly difference in the natural log of the exchange rate, I created a new column where all values have been shifted down one row
merged_df["Shifted French CPI"] = merged_df["France: Prices, Consumer Price Index"].shift(1)
#I then subtracted these shifted values from the original data column
merged_df["(iii) Monthly inflation rate"] = (merged_df["France: Prices, Consumer Price Index"] - merged_df["Shifted French CPI"])/(merged_df["France: Prices, Consumer Price Index"])

In [ ]:
merged_df['(iv) France: Natural log of Industrial Production'] = np.log(merged_df['France: Economic Activity, Industrial Production, Index'])    
#created a new column where we have taken logs of each value in the industrial production column

In [ ]:
#Following a similar process to the monthly difference in the natural log of the exchange rate, I created a new column where all values have been shifted down one row
merged_df["Shifted French Industrial production"] = merged_df["(iv) France: Natural log of Industrial Production"].shift(1)
#I then subtracted these shifted values from the original data column
merged_df["(iii) Monthly growth in industrial production"] = merged_df["(iv) France: Natural log of Industrial Production"] - merged_df["Shifted French Industrial production"]

In [ ]:
#Following a similar process to the monthly difference in the natural log of the exchange rate, I created a new column where all values have been shifted down 12 rows
merged_df["Shifted 12 rows French Industrial production"] = merged_df["(iv) France: Natural log of Industrial Production"].shift(12)
#I then subtracted these shifted values from the original data column
merged_df["(vi) France: Industrial production growth vs previous year"] = merged_df["(iv) France: Natural log of Industrial Production"] - merged_df["Shifted 12 rows French Industrial production"]

In [ ]:
merged_df['(i) US: Natural log of CPI'] = np.log(merged_df['United States: Prices, Consumer Price Index'])    
#Taking the natural log of the US CPI

In [ ]:
#Following a similar process to the monthly difference in the natural log of the exchange rate, I created a new column where all values have been shifted down one row
merged_df["Shifted US natural log CPI"] = merged_df["United States: Prices, Consumer Price Index"].shift(1)
#I then subtracted these shifted values from the original data column
merged_df["(iii) Monthly difference in the inflation rate"] = merged_df["United States: Prices, Consumer Price Index"] - merged_df["Shifted US natural log CPI"]

In [ ]:
stripped_df = merged_df.drop(['Shifted Natural log of the Exchange Rate per U.S. Dollar',
                             'Shifted French CPI', 
                             'Shifted French Industrial production', 
                             'Shifted 12 rows French Industrial production', 
                             'Shifted US natural log CPI'], 
                            axis=1)
stripped_df
#This step is to strip away all these extra columns. 
#These columns are the colunmns where all the data values have been shifted down
#They were used as part of the process to find monthly differences
#However, now that we have the final column, we do not require these and keeping them would clutter the dataframe.

In [ ]:
stripped_df.isnull().sum() 
# This is to check the number of missing values in the dataset.

In [ ]:
def outlier_limits(col):
    Top99_5, Lowest0_5 = np.nanpercentile(col, [99.5, 0.5])
    Upper_limit = Top99_5
    Lower_limit = Lowest0_5
    return Upper_limit, Lower_limit

#We defined top 0.5% and bottom 0.5% as outliers

In [ ]:
for column in stripped_df.columns:
    if stripped_df[column].dtype != 'object':
        Upper_limit, Lower_limit = outlier_limits(stripped_df[column])
        stripped_df[column] = np.where((stripped_df[column] > Upper_limit)| (stripped_df[column] < Lower_limit), np.nan, stripped_df[column])
        
 # We want to make these outliers missing value

In [ ]:
stripped_df.isnull().sum()
# this tells us the number of missing values after converting outliers into missing value. 
# The difference in missing value after and before tells us the number of outliers identified.

In [ ]:
new_stripped_df = stripped_df.interpolate()
new_stripped_df.isnull().sum()
#Using linear interpolation, we can fill in the missing values.
#Checking the number of missing values after linear interpolation.

In [ ]:
#To construct a series for monthly real exchange rate growth, where the real exchange rate is measured for your country relative to the united states
#creating a new column for real exchange rates. formula from LT 5 is nominal*price in US / price in France
new_stripped_df['Nominal exchange rate * US prices'] = new_stripped_df['France: Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate'] * new_stripped_df['United States: Prices, Consumer Price Index']
new_stripped_df['Monthly real exchange rate'] = new_stripped_df['Nominal exchange rate * US prices'] / new_stripped_df['France: Prices, Consumer Price Index']


In [ ]:
#France monthly exchange rate growth

new_stripped_df['France: Monthly real exchange rate growth']= np.log(new_stripped_df['Monthly real exchange rate']) - np.log(new_stripped_df['Monthly real exchange rate']).shift(1)

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib.dates import YearLocator, DateFormatter
f, ax = plt.subplots(figsize = [12, 7])
tempdf = pd.DataFrame({"val": list(range(372))})
tempdf["dates"] = [datetime(1960,1,31) + relativedelta(months=x) for x in range(0,372)]
x = tempdf["dates"]
y = new_stripped_df["(iii) Monthly growth in industrial production"]
plt.scatter(x, y)
ax.xaxis.set_major_locator(YearLocator(5))
#labels every 5 years on x axis to not clutter
ax.xaxis.set_major_formatter(DateFormatter("%Y"))

In [ ]:
(new_stripped_df)

In [ ]:
f, ax = plt.subplots(figsize = [12, 7])
plt.title("Monthly Nominal Exchange Rate Growth")
plt.xlabel("Year")
plt.ylabel("Monthly Nomianl Exchange Rate Growth (%)")
ax.plot(tempdf["dates"], new_stripped_df['(ii) Monthly difference, Natural log of the Exchange Rate per U.S. Dollar'])
nobretton = datetime(1971,8,15)
ax.vlines(nobretton,-0.1, 0.1, linestyles='dashed', colors='red',label = "France left BW")
#date Bretton Woods ended
ax.xaxis.set_major_locator(YearLocator(5))
#labels every 5 years on x axis to not clutter
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend()



In [ ]:
f, ax = plt.subplots(figsize = [12, 7])
plt.title("Monthly Real Exchange Rate Growth")
plt.xlabel("Year")
plt.ylabel("Monthly Nomianl Exchange Rate Growth (%)")
ax.plot(tempdf["dates"], new_stripped_df['France: Monthly real exchange rate growth'])
nobretton = datetime(1971,8,15)
ax.vlines(nobretton,-0.1, 0.1, linestyles='dashed', colors='red',label = "France left BW")
#date Bretton Woods ended
ax.xaxis.set_major_locator(YearLocator(5))
#labels every 5 years on x axis to not clutter
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend()



In [ ]:
f, ax = plt.subplots(figsize = [12, 7])
plt.title("Monthly Inflation Rate")
plt.xlabel("Year")
plt.ylabel("Inflation rate(%)")
ax.plot(tempdf["dates"], new_stripped_df['(iii) Monthly difference in the inflation rate'])
ax.vlines(nobretton,-0.2, 0.6, linestyles='dashed', colors='red',label = "France left BW")
ax.xaxis.set_major_locator(YearLocator(5))
#labels every 5 years on x axis to not clutter
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend()

In [ ]:
f, ax = plt.subplots(figsize = [12, 7])
plt.title("Monthly Industrial Production Growth")
plt.xlabel("Year")
plt.ylabel("Industrial production growth (%)")
ax.plot(tempdf["dates"], new_stripped_df['(iii) Monthly growth in industrial production'])
ax.vlines(nobretton,-0.5, 0.6, linestyles='dashed', colors='red',label = "France left BW")
ax.xaxis.set_major_locator(YearLocator(5))
#labels every 5 years on x axis to not clutter
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend()

In [ ]:
f, ax = plt.subplots(figsize = [12, 7])
plt.title("Yearly Industrial Production Growth")
plt.xlabel("Year")
plt.ylabel("Yearly Industrial Production Growth (%)")
ax.plot(tempdf["dates"], new_stripped_df['(vi) France: Industrial production growth vs previous year'])
ax.vlines(nobretton,-0.2, 0.3, linestyles='dashed', colors='red',label = "France left BW")
ax.xaxis.set_major_locator(YearLocator(5))
#labels every 5 years on x axis to not clutter
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend()

In [ ]:
new_stripped_df[0:127].std(skipna=True, numeric_only=True)
#standard deviations of before leaving bretton woods excluding year before

In [ ]:
new_stripped_df[152:].std(skipna=True, numeric_only=True)
#standard deviations of after leaving bretton woods excluding year after

In [ ]:
final_df = new_stripped_df.drop(['Nominal exchange rate * US prices'], axis=1)

In [ ]:
final_df.rename(columns={'France: Economic Activity, Industrial Production, Index': 'France: Industrial Production',
                         'France: Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate': 'France: Nominal ER, USD',
                         'France: Prices, Consumer Price Index': 'France: CPI',
                         'United States: Prices, Consumer Price Index': 'US: CPI',
                         '(i) France: Natural log of the Exchange Rate per U.S. Dollar': '(i) France: Nlog ER, USD',
                         '(ii) Monthly difference, Natural log of the Exchange Rate per U.S. Dollar': '(ii) France: Monthly difference, Nlog ER, USD',
                         '(iv) France: Natural log of Industrial Production': '(iv) France: Nlog Industrial Production',
                         '(iii) Monthly growth in industrial production': '(v) France: Monthly growth in industrial production',
                         '(i) US: Natural log of CPI': 'US: Nlog CPI',
                         'Monthly real exchange rate': 'France: Monthly real ER'}, inplace=True)

#Note that 'ER' stands for Exchange rate
#Note that 'Nlog' stands for Natural log of
#This step was just to rename some lengthy column names which were a bit of a mouthful.

In [ ]:
final_df.index = final_df['Month']
final_df = final_df.drop(['Month'], axis=1)
final_df

In [ ]:
final_df['France-US difference in inflation'] = final_df['(iii) Monthly inflation rate'] - final_df['(iii) Monthly difference in the inflation rate']
#Dataset for 
final_dfBeforeLeaving = final_df.loc['Jan 1960':'Aug 1970']
final_dfAfterLeaving = final_df.loc['Aug 1972':'Dec 1990']
BrettonWoods_variables = ['(ii) France: Monthly difference, Nlog ER, USD','France: Monthly real exchange rate growth','(ii) France: Monthly difference, Nlog ER, USD','France-US difference in inflation','(vi) France: Industrial production growth vs previous year']
x = final_dfBeforeLeaving[BrettonWoods_variables].std()
y = final_dfAfterLeaving[BrettonWoods_variables].std()
q=x/y
results = pd.DataFrame({"BEFORE":[],"AFTER":[],"RATIO":[]})
for i in range(len(BrettonWoods_variables)):
    results.loc[BrettonWoods_variables[i]] = [x[i],y[i],q[i]] 
results


